## Junta tablas

In [14]:
import os
import pandas as pd
import re
from datetime import datetime

In [15]:
# SELECCIONAR QUÉ PRODUCTO QUEREMOS AGRUPAR
producto = "iphone"

In [16]:
def read_and_join_csvs_by_date(fecha, producto, folder_path='1_datos_raw'):
    """
    Función que toma una fecha en formato 'yyyymmdd' y busca los archivos CSV 
    en el folder_path que comiencen con esa fecha, añadiendo la fecha como columna
    y realizando un outer join entre ellos.

    Args:
    -(fecha) (str): Fecha en formato 'yyyymmdd'.
    - folder_path (str): Directorio donde se encuentran los archivos CSV. Por defecto, es el directorio actual.

    Returns:
    - pd.DataFrame: DataFrame resultante del outer join entre los CSV encontrados con la misma fecha,
                    incluyendo una columna "fecha".
    """
    # Listar todos los archivos en la carpeta
    files = os.listdir(folder_path)
    
    # Filtrar los archivos que comienzan con la fecha proporcionada
    csv_files = []
    for f in files:
        if f.startswith(fecha) and f.endswith('.csv'):
            csv_files.append(f)
    product_files = []
    for f in csv_files:
        if producto in f.lower():
            product_files.append(f)

    print(product_files)
    # Verificar que haya más de un archivo para realizar el join
    if len(product_files) > 1:
        # Leer los archivos con la misma fecha y añadir la columna "fecha"
        dfs = []
        for f in product_files:
            df = pd.read_csv(os.path.join(folder_path, f))
            df['fecha'] =(fecha)  # Añadir la columna con la fecha
            dfs.append(df)

        # Hacer un join de todos los DataFrames de esa fecha usando outer join
        merged_df = dfs[0]
        for df in dfs[1:]:
            merged_df = merged_df.merge(df, how='outer')  # Outer join para añadir registros no coincidentes
        
        return merged_df
    else:
        print(f"No se encontraron suficientes archivos con la fecha {fecha} para hacer un join.")
        return None

# Uso de la función
fecha = datetime.now().strftime("%Y%m%d")
fecha = '20241114'  # Fecha en formato 'yyyymmdd'
df = read_and_join_csvs_by_date(fecha, producto)

## Para arreglar los actuales
df['scrap_locat'] = 'madrid'
df['dist_scrap_locat'] = 60
column_order = df.columns.tolist()
column_order.insert(5, column_order.pop(column_order.index('scrap_locat')))
column_order.insert(6, column_order.pop(column_order.index('dist_scrap_locat')))
df = df[column_order]
########

del df['Unnamed: 0']
df

['20241114_iphone 13_new.csv', '20241114_iphone 14_as_good_as_new.csv', '20241114_iphone 14_good.csv', '20241114_iphone 14_new.csv', '20241114_iphone 15_as_good_as_new.csv', '20241114_iphone 15_good.csv', '20241114_iphone 15_new.csv', '20241114_iphone 16_as_good_as_new.csv', '20241114_iphone 16_good.csv', '20241114_iphone 16_new.csv']


,id,nombre,precio,estado,scrap_locat,dist_scrap_locat,reservado,url,fecha
0,1068541305,iphone 16 128gb,"800,00",new,madrid,60,False,https://es.wallapop.com/item/iphone-16-128gb-1...,20241114
1,1069277594,iphone 14 pro morado,"700,00",new,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-mor...,20241114
2,1069385453,iphone 16,"850,00",as_good_as_new,madrid,60,False,https://es.wallapop.com/item/iphone-16-1069385453,20241114
3,1069408278,iphone 14 pro,"800,00",good,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-106...,20241114
4,1069426273,iphone 16 pro max,"1020,00",good,madrid,60,False,https://es.wallapop.com/item/iphone-16-pro-max...,20241114
...,...,...,...,...,...,...,...,...,...
1526,1064280078,iphone 14 plus 128gb,"589,00",as_good_as_new,madrid,60,False,https://es.wallapop.com/item/iphone-14-plus-12...,20241114
1527,1063248685,iphone 14 pro,"675,00",as_good_as_new,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-106...,20241114
1528,1063827942,iphone 14 pro max,"850,00",as_good_as_new,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-max...,20241114
1529,1064909050,iphone 14 pro max 128gb nuevo,"799,00",as_good_as_new,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-max...,20241114


In [17]:
if producto == "iphone":
    # Creo una columna en la que pongo que iphone es (14, 15 o 16..) y que tipo. 
    # Gen
    # Model
    for index in range(len(df)):
        df['gen'] = 'vacio'
        df['modelo'] = 'modelo'

    def asignar_gen(nombre):
        if '14' in nombre:
            return '14'
        elif '15' in nombre:
            return '15'
        elif '16' in nombre:
            return '16'
        return "n/a"

    # Función para asignar el valor de la columna "modelo"
    def asignar_modelo(nombre):
        if 'pro max' in nombre.lower():
            return 'pro max'
        elif 'pro' in nombre.lower():
            return 'pro'
        elif 'plus' in nombre.lower():
            return 'plus'
        else:
            return 'basic'

    def asignar_memoria(nombre):
        if '128' in nombre.lower():
            return '128'
        elif '256' in nombre.lower():
            return '256'
        elif '512' in nombre.lower():
            return '512'
        elif '1tb' in nombre.lower():
            return '1tb'
        else:
            return 'n/a'

    # Función para asignar el valor de la columna "bateria"
    def asignar_bateria(nombre):
        # Buscar números entre 80 y 100 en el nombre
        bateria = re.findall(r'\b([8-9][0-9]|100)\b', nombre)
        if bateria:
            return int(bateria[0])  # Retorna el primer número que encuentra
        return "n/a"

    # Aplicar las funciones al DataFrame
    df['gen'] = df['nombre'].apply(asignar_gen)
    df['mod'] = df['nombre'].apply(asignar_modelo)
    df['memoria'] = df['nombre'].apply(asignar_memoria)
    df['bateria'] = df['nombre'].apply(asignar_bateria)
    orden_columnas = ['id','fecha', 'gen', 'mod', 'memoria', 'bateria','precio', 'estado', 'nombre', 'scrap_locat' , 'dist_scrap_locat',
     'reservado', 'url']
    df = df[orden_columnas]
    df

In [18]:
carpeta = "2_datos_por_fecha"
nombre_archivo_pkl = carpeta + '/' + producto + '_' + fecha + '.csv'
df.to_csv(nombre_archivo_pkl)
print(nombre_archivo_pkl)

2_datos_por_fecha/iphone_20241114.csv
